In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA1
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA2
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sankrandanloke/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
def find_btc_webpage_appearances(btc_address):
  search_url = "https://www.bitcoinwhoswho.com/address/" + btc_address

  # f = urlopen(search_url)
  # raw_content = f.read()
  raw_content = get_content(search_url)

  soup = BeautifulSoup(raw_content, 'html.parser')
  website_appearances_raw = soup.find_all('div', id="url_bitcoin_found_table")

  webpage_appearances = []

  if(len(website_appearances_raw) != 0):
    website_appearances_raw_html = website_appearances_raw[0]
    # print(website_appearances_raw_html)
    for link in website_appearances_raw_html.find_all('a', href = True):
      webpage_appearances.append(link['href'])

  return webpage_appearances

def remove_tags(html):
  soup = BeautifulSoup(html, "html.parser")

  for data in soup(['style', 'script']):
    data.decompose()

  return ' '.join(soup.stripped_strings)

def title_SA(webpage_appearances, btc_address):
  freq_count = []
  title_sa = []
  urls = []
  for link in range(len(webpage_appearances)):
    content = get_content(webpage_appearances[link])
    if content == 0:
      continue
    soup = BeautifulSoup(content, 'html.parser')
    if soup.title is not None:
      freq_count.append(btc_addr_frequency(content, btc_address))
      title_sa.append(sentimentAnalyzer1(soup.title.text))
      urls.append(webpage_appearances[link])
  return freq_count, title_sa, urls

def list_neg_websites(btc_address):
  webpage_appearances = find_btc_webpage_appearances(btc_address)
  freqs, title_sa, urls = title_SA(webpage_appearances, btc_address)
  resultSet = []
  # print('Url, Positive, Negative, Frequency')
  for item in range(len(title_sa)):
    # if (title_sa[item]['neg'] > title_sa[item]['pos']): # if negative score > positive score
    #   print(freqs[item], ' --- ', urls[item])
    resultSet.append({'url':urls[item],'pos':title_sa[item]['pos'],'neg':title_sa[item]['neg'],'freq':freqs[item]})
  # print(resultSet)
  return resultSet
    
def sentimentAnalyzer1(sentence):
    sid = SIA1()
    score1 = sid.polarity_scores(sentence)
    return score1
def sentimentAnalyzer2(sentence):
    vs = SIA2()
    score2 = vs.polarity_scores(sentence)
    return score2

def btc_addr_frequency(html, btc_address):
  # text = remove_tags(html)
  text = html.decode('ISO-8859-1')
  indices = [i for i in range(len(text)) if text.startswith(btc_address, i)]
  return len(indices)

def get_content(link):
  try:
    req = Request(link, headers={'User-Agent':'Mozilla/5.0'})
    content = urlopen(req).read()
    return content
  except Exception as e:
    print('Exception details: ', e, '\nLink: ', link, '\n')
    return 0

In [3]:
btc_address = '12t9YDPgwueZ9NyMgw519p7AA8isjr6SMw'
list_neg_websites(btc_address)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Exception details:  <urlopen error [Errno 61] Connection refused> 
Link:  https://arhivach.net/thread/261173/ 

Exception details:  <urlopen error [Errno 61] Connection refused> 
Link:  https://arhivach.net/thread/261420/ 

Exception details:  <urlopen error [Errno 61] Connection refused> 
Link:  https://arhivach.net/thread/261563/ 

Exception details:  <urlopen error [Errno 61] Connection refused> 
Link:  https://arhivach.net/thread/261857/ 

Exception details:  HTTP Error 403: Forbidden 
Link:  https://www.cryptocoinsnews.com/nearly-53k-bitcoin-ransoms-paid-wannacry-ransomware/ 

Exception details:  HTTP Error 503: Service Temporarily Unavailable 
Link:  https://bitcointalk.org/index.php?topic=1916199.0;all 



[{'url': 'https://www.pcrisk.com/removal-guides/20077-niros-ransomware',
  'pos': 0.0,
  'neg': 0.0,
  'freq': 2},
 {'url': 'https://www.pcrisk.com/removal-guides/19843-fbi-screenlocker',
  'pos': 0.0,
  'neg': 0.0,
  'freq': 2},
 {'url': 'https://wanna-cry-profits.herokuapp.com/',
  'pos': 0.744,
  'neg': 0.0,
  'freq': 2},
 {'url': 'https://www.recordedfuture.com/wannacry-ransomware-analysis/',
  'pos': 0.0,
  'neg': 0.307,
  'freq': 2},
 {'url': 'https://www.crowdstrike.com/blog/falcon-intelligence-report-wanna-ransomware-spreads-rapidly-continually-encrypts-victim-files/',
  'pos': 0.138,
  'neg': 0.33,
  'freq': 1},
 {'url': 'https://malware.news/t/an-analysis-of-the-wannacry-ransomware-outbreak/11908',
  'pos': 0.0,
  'neg': 0.0,
  'freq': 1},
 {'url': 'https://sensorstechforum.com/ooops-important-files-encrypted/',
  'pos': 0.231,
  'neg': 0.0,
  'freq': 1},
 {'url': 'https://www.schneier.com/blog/archives/2017/05/the_future_of_r.html',
  'pos': 0.286,
  'neg': 0.0,
  'freq': 2}